# InitialSetup

In [1]:
import os

if not os.path.exists('data_set/'):
    raise Exception("Generated dataset not found, pls run 'SetupDataset' first.")
    
if not os.path.exists('OUTPUT/'):
    os.makedirs('OUTPUT/')

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

2023-09-02 12:34:22.876125: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Loading dataset

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255, brightness_range=[0.2,1.0], shear_range=0.2,
                                    fill_mode = 'nearest', width_shift_range=0.2, rotation_range=40,
                                   height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2)
training_data = train_gen.flow_from_directory('data_set/train', target_size=(256,256), 
                                                 batch_size=32, class_mode='binary')

val_gen = ImageDataGenerator(rescale=1./255)
val_data = val_gen.flow_from_directory('data_set/val', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory('data_set/test', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

Found 2485 images belonging to 2 classes.
Found 532 images belonging to 2 classes.
Found 535 images belonging to 2 classes.


# Training

In [4]:
IMAGE_SHAPE = (256,256,3)
validation_steps = len(val_data)
steps_per_epoch = len(training_data)

In [5]:
def tune_model(pretrain_model):
    classifier = Sequential([
        pretrain_model,
    
        GlobalAveragePooling2D(),
    
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
    
    classifier.fit(training_data, epochs=30, steps_per_epoch=steps_per_epoch, workers=1,
                    validation_data=val_data, validation_steps=validation_steps)
    
    return classifier

## ResNet

In [6]:
from tensorflow.keras.applications import ResNet152V2

res_net = ResNet152V2(input_shape=IMAGE_SHAPE, include_top=False, weights='imagenet')

In [7]:
resnet_classifier = tune_model(res_net)

Epoch 1/30


2023-09-02 12:34:33.259976: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


78/78 [==============================] - ETA: 0s - loss: 0.2335 - accuracy: 0.9082 

2023-09-02 13:10:59.360242: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


78/78 [==============================] - 2288s 29s/step - loss: 0.2335 - accuracy: 0.9082 - val_loss: 761.9999 - val_accuracy: 0.6955
Epoch 2/30
78/78 [==============================] - 2335s 30s/step - loss: 0.1112 - accuracy: 0.9630 - val_loss: 1.3463 - val_accuracy: 0.9117
Epoch 3/30
78/78 [==============================] - 2260s 29s/step - loss: 0.0954 - accuracy: 0.9686 - val_loss: 134.6133 - val_accuracy: 0.6974
Epoch 4/30
78/78 [==============================] - 2263s 29s/step - loss: 0.0975 - accuracy: 0.9690 - val_loss: 4.2889 - val_accuracy: 0.7086
Epoch 5/30
78/78 [==============================] - 2137s 27s/step - loss: 0.0631 - accuracy: 0.9751 - val_loss: 0.2055 - val_accuracy: 0.9643
Epoch 6/30
78/78 [==============================] - 2111s 27s/step - loss: 0.0605 - accuracy: 0.9807 - val_loss: 1.8950 - val_accuracy: 0.7932
Epoch 7/30
78/78 [==============================] - 2515s 32s/step - loss: 0.0582 - accuracy: 0.9771 - val_loss: 0.1219 - val_accuracy: 0.9568
Epoch 

In [8]:
resnet_classifier.save("OUTPUT/ResnetModel.h5")

In [9]:
_, accuracy = resnet_classifier.evaluate(test_data, steps = validation_steps)

2023-09-03 06:40:13.955045: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


17/17 [==============================] - 94s 6s/step - loss: 0.0535 - accuracy: 0.9907


In [10]:
print('ResnetModel accuracy :',accuracy)

ResnetModel accuracy : 0.9906542301177979
